## ScriptQueue Miner ##

An illustration of using the ScriptQueue query tools, which are behind the "SQ Miner" notebook on Times Square and the Context Feed on Nightly Digest.

In [ ]:
import os
import numpy as np
import pandas as pd 

import logging
logging.getLogger('rubin_nights').setLevel(logging.INFO)

from astropy.time import Time, TimeDelta

from IPython.display import display, HTML
from rubin_nights import connections
from rubin_nights import scriptqueue
from rubin_nights import scriptqueue_formatting
from rubin_nights.ts_xml_enums import CategoryIndexExtended

In [ ]:
# Test if running in an RSP - if so, token and site will be automatic
if os.getenv("EXTERNAL_INSTANCE_URL") is None:
    # Outside of an RSP, you will need an RSP token
    # See https://rsp.lsst.io/guides/auth/creating-user-tokens.html 
    site = "usdf"
    tokenfile = os.path.join(os.path.expanduser("~"), ".lsst/usdf_rsp")
else:
    tokenfile = None
    site = None

print(f"Using site {site}")
endpoints = connections.get_clients(tokenfile=tokenfile, site=site)

In [ ]:
from zoneinfo import ZoneInfo
tz = ZoneInfo("Chile/Continental")
tz_utc = ZoneInfo("UTC")
tz_pacific = ZoneInfo("America/Los_Angeles")

t = Time.now()
print("UTC time", t.utc.to_datetime(timezone=tz_utc))
print("Chilean time", t.utc.to_datetime(timezone=tz))
print("Pacific time", t.utc.to_datetime(timezone=tz_pacific))

In [ ]:
# Search based on dayobs?
day_obs = "2025-12-07"
t_start = Time(f"{day_obs}T12:00:00", format='isot', scale='utc')
t_end = t_start + TimeDelta(1, format='jd')

# Search based on current time and previous sunset to now?
# from astroplan import Observer
# observer = Observer.at_site('Rubin')
# t_start = observer.sun_set_time(Time.now(), which='previous')
# t_end = Time.now

#Search for the last few hours 
#t_start = Time.now() - TimeDelta(5*60*60, format='sec')
#t_end = Time.now()

print(f"Querying for messages from {t_start.iso} to {t_end.iso}")
print(f"Notebook executed at {Time.now().utc.iso}")

efd_and_messages, cols = scriptqueue.get_consolidated_messages(t_start, t_end, endpoints, all_tracebacks=True)

In [ ]:
mt = [i.value for i in CategoryIndexExtended if "SIMONYI" in i.name or "OTHER" in i.name or "MAIN" in i.name or "OCS" in i.name]
at = [i.value for i in CategoryIndexExtended if "AUX" in i.name or "OTHER" in i.name ]
simonyi = scriptqueue_formatting.format_html(efd_and_messages, cols=cols, time_order='newest first', show_category_index=mt)
auxtel = scriptqueue_formatting.format_html(efd_and_messages, cols=cols, time_order='newest first', show_category_index=at)
both = scriptqueue_formatting.format_html(efd_and_messages, cols=cols, time_order='oldest first')
display(HTML(simonyi))

In [ ]:
# t_start = Time("2025-11-26 06:29:56.342671", scale='utc')
# t_end = Time.now()
# mm = endpoints['efd'].select_time_series("lsst.sal.Scheduler.logevent_logMessage", '*', t_start, t_end, index=1)
# display(HTML(mm[['functionName', 'message']].to_html()))